In [1]:
'''
!pip install "numpy<2.0.0"
'''

'\n!pip install "numpy<2.0.0"\n'

In [2]:
import gymnasium as gym
from gymnasium import envs
import numpy as np
from collections import deque
import random
import matplotlib.pyplot as plt
import cv2 
import time
import warnings
import ale_py
import multiprocessing as mp
from datetime import datetime
import os
import pandas as pd
from keras.models import Sequential, clone_model
from keras.layers import Conv2D, Flatten, Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from collections import deque
import random
import math

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.4.0) or chardet (4.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Please experiment with the numbers, we want to make it to the end, and I know it'll take a while so please do not randomly choose

In [3]:
# --- Hyperparameters ---
EPISODES = 3000                
BATCH_SIZE = 32                
MEMORY_SIZE = 100000           
GAMMA = 0.99                   
EPSILON_START = 0.5            
EPSILON_MIN = 0.01             
EPSILON_DECAY = 0.995          
LEARNING_RATE = 0.00025        
UPDATE_TARGET_FREQ = 1000      
FRAME_STACK = 4                
SAVE_FREQ = 50                 

# --- Prioritized Experience Replay (PER) ---
class PrioritizedReplayBuffer:
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = deque(maxlen=max_size)
        self.priorities = deque(maxlen=max_size)
    
    def add(self, state, action, reward, next_state, done):
        max_priority = max(self.priorities) if self.buffer else 1.0
        self.buffer.append((state, action, reward, next_state, done))
        self.priorities.append(max_priority)
    
    def sample(self, batch_size, alpha=0.6):
        priorities = np.array(self.priorities)
        probs = priorities ** alpha
        probs /= probs.sum()
        
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[i] for i in indices]
        weights = (len(self.buffer) * probs[indices]) ** (-0.4)
        weights /= weights.max()
        
        states = np.array([x[0] for x in samples])
        actions = np.array([x[1] for x in samples])
        rewards = np.array([x[2] for x in samples])
        next_states = np.array([x[3] for x in samples])
        dones = np.array([x[4] for x in samples])
        
        return states, actions, rewards, next_states, dones, indices, weights
    
    def update_priorities(self, indices, errors, offset=0.01):
        for i, error in zip(indices, errors):
            self.priorities[i] = abs(error) + offset

# --- Environment Setup ---
env = gym.make("ALE/Frogger-v5", render_mode="rgb_array")
state_shape = (84, 84, FRAME_STACK)
action_size = env.action_space.n

# --- Model Definition ---
def build_model():
    model = Sequential([
        Conv2D(32, (8, 8), strides=4, activation="relu", input_shape=state_shape),
        Conv2D(64, (4, 4), strides=2, activation="relu"),
        Conv2D(64, (3, 3), strides=1, activation="relu"),
        Flatten(),
        Dense(512, activation="relu"),
        Dense(action_size, activation="linear")
    ])
    model.compile(loss="mse", optimizer=Adam(learning_rate=LEARNING_RATE))
    return model

# --- Frame Stacking ---
class FrameStacker:
    def __init__(self):
        self.frames = deque(maxlen=FRAME_STACK)
    
    def reset(self, state):
        for _ in range(FRAME_STACK):
            self.frames.append(state)
        return np.stack(self.frames, axis=-1)
    
    def append(self, state):
        self.frames.append(state)
        return np.stack(self.frames, axis=-1)

# --- Preprocessing ---
def preprocess_state(state):
    state = np.mean(state, axis=2)  
    state = state[34:194, :]        
    state = state[::2, ::2]         
    state = np.pad(state, ((2,2),(2,2)), mode='constant')
    return state / 255.0

# --- Training Functions ---
def load_model_weights(model, weights_path):
    if weights_path:
        try:
            model.load_weights(weights_path)
            print(f"Successfully loaded weights from {weights_path}")
            return True
        except Exception as e:
            print(f"Could not load weights from {weights_path}: {str(e)}")
            return False
    return False

def train_dqn(weights_path=None):  # Add weights_path parameter
    model = build_model()
    target_model = clone_model(model)
    
    # Load weights if provided
    if weights_path:
        try:
            model.load_weights(weights_path)
            print(f"Successfully loaded weights from {weights_path}")
        except:
            print(f"Could not load weights from {weights_path}, starting from scratch")
    
    target_model.set_weights(model.get_weights())
    
    memory = PrioritizedReplayBuffer(MEMORY_SIZE)  
    frame_stacker = FrameStacker()
    epsilon = EPSILON_START
    rewards_history = []
    start_time = time.time()
    global_step = 0

    for episode in range(1, EPISODES + 1):
        state, _ = env.reset()
        state = preprocess_state(state)
        state = frame_stacker.reset(state)
        total_reward = 0
        done = False

        while not done:
            global_step += 1

            if np.random.rand() <= epsilon:
                action = env.action_space.sample()
            else:
                q_values = model.predict(np.expand_dims(state, axis=0), verbose=0)
                action = np.argmax(q_values[0])

            next_state, reward, done, _, _ = env.step(action)
            next_state = preprocess_state(next_state)
            next_state = frame_stacker.append(next_state)
            memory.add(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

            if global_step % 4 == 0 and len(memory.buffer) >= BATCH_SIZE:
                states, actions, rewards, next_states, dones, indices, weights = memory.sample(BATCH_SIZE)
                
                next_q_values = target_model.predict(next_states, verbose=0)
                best_actions = np.argmax(model.predict(next_states, verbose=0), axis=1)
                target_q = rewards + GAMMA * next_q_values[np.arange(BATCH_SIZE), best_actions] * (1 - dones)
                
                current_q = model.predict(states, verbose=0)
                td_errors = target_q - current_q[np.arange(BATCH_SIZE), actions]
                memory.update_priorities(indices, td_errors)
                
                target = current_q.copy()
                target[np.arange(BATCH_SIZE), actions] = target_q
                model.fit(states, target, sample_weight=weights, epochs=1, verbose=0)

            if global_step % UPDATE_TARGET_FREQ == 0:
                target_model.set_weights(model.get_weights())

        if epsilon > EPSILON_MIN:
            epsilon *= EPSILON_DECAY

        rewards_history.append(total_reward)

        elapsed_time = (time.time() - start_time) / 60  # in minutes
        remaining_time = (elapsed_time / episode) * (EPISODES - episode)
        print(f"Episode: {episode}/{EPISODES}, Reward: {total_reward}, Epsilon: {epsilon:.2f}, Time Elapsed: {elapsed_time:.2f} mins, Remaining: {remaining_time:.2f} mins")

        if episode % SAVE_FREQ == 0:
            model.save_weights(f"frogger_weights_ep{episode}.weights.h5")
            print(f"Saved weights at episode {episode}")

    plt.plot(rewards_history)
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.title("Frogger DQN Training (PER + Double DQN + Frame Stack)")
    plt.savefig("frogger_training_enhanced.png")
    plt.show()

    return model

# --- Run Training ---
if __name__ == "__main__":
    # Specify the path to your existing weights file (or None to start fresh)
    weights_to_load = "frogger_weights_ep2700.weights.h5"  # Change this to your weights file
    
    model = train_dqn(weights_path=weights_to_load)
    model.save_weights("frogger_final_weights_enhanced.weights.h5")

C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\wreep\AppData\Roaming\Python\Python39\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Successfully loaded weights from frogger_weights_ep2700.weights.h5
Episode: 1/3000, Reward: 9.0, Epsilon: 0.50, Time Elapsed: 0.42 mins, Remaining: 1267.89 mins
Episode: 2/3000, Reward: 12.0, Epsilon: 0.50, Time Elapsed: 0.84 mins, Remaining: 1263.06 mins
Episode: 3/3000, Reward: 19.0, Epsilon: 0.49, Time Elapsed: 1.53 mins, Remaining: 1533.15 mins
Episode: 4/3000, Reward: 13.0, Epsilon: 0.49, Time Elapsed: 2.12 mins, Remaining: 1587.77 mins
Episode: 5/3000, Reward: 17.0, Epsilon: 0.49, Time Elapsed: 2.66 mins, Remaining: 1594.31 mins
Episode: 6/3000, Reward: 12.0, Epsilon: 0.49, Time Elapsed: 3.10 mins, Remaining: 1547.46 mins
Episode: 7/3000, Reward: 18.0, Epsilon: 0.48, Time Elapsed: 3.57 mins, Remaining: 1526.05 mins
Episode: 8/3000, Reward: 14.0, Epsilon: 0.48, Time Elapsed: 4.00 mins, Remaining: 1495.28 mins


KeyboardInterrupt: 